# ECON 1923 CPI Project
## Created by: Gloria Dou, Peter Martin, and Kavya Singh

## 2 Definition of the Index
### 2.1 Items and weights

Products, services and their mix should be decided. Your basket must include at least 8 ingredients. Your choice should be justified. Use the online and other resources to build a reasonable basket of goods that represents, in your opinion, students. There are no correct or incorrect answers, I am looking for a well justified answer.

### 2.2 Locations and weights
The locations where these prices are measured. List at least three locations based on where the team members either live now, lived in the past, or want to live in the future.2 Choose weights for these locations according to your liking.

### 2.3 Finalizing the basket
Combining the choices of items and locations, find all the series names assign a weight for each series that combines the weights that you chose for the items and the weights you chose for the locations.



https://docs.google.com/document/d/1Rb8_QNXB3ybjHzDLKQBadFX2rS7RIILdCIdtplUyxg4/edit

- Everyone’s github User ID:
    - Kavya: kavyas01
    - Gloria: GloDDD
    - Peter: mpaertteirn

- Group of products, services, etc. that students would use (minimum 8): 
    - Fruits and Vegetables (SAF113) [0.20]
    - Meats (SAF11211) [0.20]
    - Rent of Primary  Residence (SEHA) [0.20]
    - Coffee (SEFP01) [0.15]
    - Smartphones (SSEE041) [0.10]
    - Apparel (SAA) [0.10]
    - College Textbooks (SSEA011) [0.04]
    - Public Transportation (SETG) [0.01]

- Three locations: 
    - Pittsburgh  (A104) [0.50]
    - Philadelphia (S12B) [0.40]
    - Orange County (A421) [0.10]

## 3 Downloading the data
### 3.1 Refine the basket
First, verify that the series you created in the previous section exist in the BLS’ API. If some items do not exist, take them out and re-weight. If a certain location produces all the missing series, replace that location with a bigger or more general location.
After you made all the changes and adjustments, state the final basket and the weights for each item in the basket. Table format will be great here.

### 3.2 Downloading
Make an API call to download the data for the basket you created. I ask that you have data for the last 5 years: January 2017 to January 2022. Make sure your API call retrieves data for this range.
    Include the general CPI (CUUR0000SA0) in your API call.
    Question: Are all the series you downloaded in a monthly frequency? If not, how can you fix that (i.e. interpolate).

## 4 Aggregating
Compute your version of the CPI based on the weights in 2.3.

## 5 Inflation
Compute the 12-month inflation for several lags based on your CPI and compare it with inflation based on the general CPI. Compare and discuss the results.
Bonus: Put both on one graph and make sure the two lines are clearly labeled as well as the axes.

## 6 Format
The group project should be in a Jupyter notebook that includes the an- swers and analysis listed above. I’m not looking for anything fancy but the following guidelines should be followed:
- Markdown should be used to create titles and to write your verbal answers to the questions.
- The names of the group members should appear at the top of the notebook.
- All variables and functions should have meaningful names (I know this is a subjective term).
- All code should be clearly and meaningfully commented, especially functions!

## 7 Grading Criteria
The project overall will be taken into account when a grade is determined. The following elements will be taken into account (including but not limited to...)
- How well explained is the process of constructing the index,
- Solving data problems (e.g. missing series, series which are not in a monthly frequency),
- Documenting the code in a clear way,
- The overall organization of the notebook,
- The discussion following the results on inflation.